### Create a GIF of Landsat images 

inspired from https://stackstac.readthedocs.io/en/latest/examples/gif.html


In [3]:
from dask.distributed import LocalCluster,Client
import dask.utils
import dask.config
import stackstac
import dask.array as da
import pystac_client
import geopandas as gpd
import planetary_computer as pc
import xarray as xr

import shapely.geometry
import os

import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from xarray.plot.utils import _rescale_imshow_rgb
import numpy as np
from typing import TYPE_CHECKING, BinaryIO, cast,Literal
import ntpath



In [4]:
# port forwarding : ssh -L 8000:localhost:8787 protect
# ou
# ssh -L 9998:129.88.193.194:22 ssh-ige
# ssh -L 9999:localhost:8787 -N -p 9998 localhost
# opne localhost:9999

cluster = LocalCluster(n_workers=10,
                       threads_per_worker=2,
                       dashboard_address=8787,
                       memory_limit='6GB')
   
#cluster = LocalCluster()   
client = Client(cluster)

display(client)

2022-10-26 11:52:15,800 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-trc5mn9w', purging
2022-10-26 11:52:15,800 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-pg4idzt8', purging
2022-10-26 11:52:15,800 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-f2o9la65', purging
2022-10-26 11:52:15,801 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-n8n1m5rg', purging
2022-10-26 11:52:15,801 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-6vkn_sk_', purging
2022-10-26 11:52:15,801 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-0jsb8qwt', purging
2022-10-26 11:52:15,801 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-sn1a3j0z', purging

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 20,Total memory: 55.88 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36871,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 55.88 GiB
Comm: tcp://127.0.0.1:45393,Total threads: 2
Dashboard: http://127.0.0.1:40089/status,Memory: 5.59 GiB
Nanny: tcp://127.0.0.1:42507,


In [5]:
from dask.utils import ensure_dict, format_bytes
    
wk = client.scheduler_info()["workers"]

text="Workers= " + str(len(wk))
memory = [w["memory_limit"] for w in wk.values()]
cores = sum(w["nthreads"] for w in wk.values())
text += ", Cores=" + str(cores)
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)

Workers= 10, Cores=20, Memory=55.88 GiB


In [51]:
def convert_bounds(bbox, invert_y=False):
    """
    Helper method for changing bounding box representation to leaflet notation
    ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))``
    """
    x1, y1, x2, y2 = bbox
    if invert_y:
        y1, y2 = y2, y1
    return ((y1, x1), (y2, x2))

# define font size for Label
def fontSize(arr):
    img = Image.fromarray(arr)
    draw = ImageDraw.Draw(img)
    txt = "Hello World"
    fontsize = 1  # starting font size
    font = ImageFont.truetype("Calibri.ttf", fontsize)
    # portion of image width you want text width to be
    img_fraction = 0.40
    breakpoint = img_fraction * img.size[0]
    jumpsize = 75
    while True:
        if font.getlength(text) < breakpoint:
            fontsize += jumpsize
        else:
            jumpsize = jumpsize // 2
            fontsize -= jumpsize
        font = ImageFont.truetype('Calibri.ttf', fontsize)
        if jumpsize <= 1:
            break

    return fontsize

def save_img(arr: xr.DataArray,
             timeStamp: bool = True,
             fontsize: int = 25,
             date_position: Literal["ul", "ur", "ll", "lr"] = "ul",
             date_color: tuple[int, int, int] = (255, 255, 255),
             date_bg: tuple[int, int, int]  = (0, 0, 0)):
        
    date_img = str(arr.time.values)[0:4]
    # Rescale
    vmin = np.nanmin(arr)
    vmax = np.nanmax(arr)
    data= _rescale_imshow_rgb(arr, vmin, vmax, True)

    # convert to U8
    u8 = (data * 255).astype("uint8").to_numpy()
    u8 = np.clip(u8, 0, 255, out=u8)
    u8 = np.moveaxis(u8, -3, -1) #??
    # Add alpha mask
    mask: np.ndarray = arr.isnull().data.any(axis=-3)
    alpha = (~mask).astype("uint8", copy=False) * 255
    frame = np.concatenate([u8, alpha[..., None]], axis=-1)

    #imgs = [Image.fromarray(frame) for frame in frames]
    img = Image.fromarray(frame)
    
     # Write timestamps onto each frame
       
    fnt =ImageFont.truetype("Calibri.ttf", fontsize)
    if timeStamp:
        
        label = 'Summer ' + date_img
        print (label)
        # get a drawing context
        d = ImageDraw.Draw(img)
        d = cast(ImageDraw.ImageDraw, d)

        width, height = img.size
        left, top, right, bottom = fnt.bbox(label)
        t_width = fnt.getlength(label)
        t_height = abs(top - bottom)
        
        offset = 50
        if date_position[0] == "u":
            y = offset
        else:
            y = height - t_height - offset

        if date_position[1] == "l":
            x = offset
        else:
            x = width - t_width - offset

        border = 8
        if date_bg:
            d.rectangle((x-border, y-border, x + t_width  + border, y + t_height+border), fill=date_bg)
        # draw text
        d.multiline_text((x, y), label, font=fnt, fill=date_color)

    
    out_filename = ntpath.basename(filename).split('.')[0]+'_'+ date_img + '.png'
    
    img.save(
        out_filename,
        format="png"
    )

In [7]:
#petermann - or create one with https://geojson.io/#map=2/20.0/0.0 and follow https://aws.amazon.com/fr/blogs/apn/transforming-geospatial-data-to-cloud-native-frameworks-with-element-84-on-aws/

#filename = "geojson file path"
filename = "petermann.geojson"
# read in AOI as a GeoDataFrame
aoi = gpd.read_file(filename)

With the pystac_client module’s Client class, Open the STAC API. 

In [11]:
datetimeRange = "2017-07-01/2017-10-15"

In [12]:
# see : https://github.com/microsoft/PlanetaryComputerExamples/blob/main/datasets/landsat-8-c2-l2/landsat-8-c2-l2-example.ipynb

bbox =aoi.unary_union.bounds

LandsatSTAC = pystac_client.Client.open('https://planetarycomputer.microsoft.com/api/stac/v1')

#for collection in LandsatSTAC.get_collections():
    #print(collection)

search = (
    LandsatSTAC
    .search(
        bbox=bbox,
        query =  {"eo:cloud_cover":{"lt":1}},
        datetime = datetimeRange, 
        collections = ["landsat-c2-l2"]
               
    )
)

In [13]:
items = pc.sign(search)
print(str(len(items))+ ' scenes found')

20 scenes found


Turn STAC items into xarray as a temporal stack, using stackstac.


In [14]:
stack = stackstac.stack(items,bounds_latlon=bbox, epsg = 32620, resolution=50)
stack

,Array,Chunk
Bytes,7.77 GiB,8.00 MiB
Shape,"(20, 19, 1618, 1696)","(1, 1, 1024, 1024)"
Count,3 Graph Layers,1520 Chunks
Type,float64,numpy.ndarray


In [15]:
# use common_name for bands
stack = stack.assign_coords(band=stack.common_name.fillna(stack.band).rename("band"))


In [16]:
# keep rgb bands + Make annual median composites (`Q` means 2 quarters)
composites = stack.sel(band=["red", "green", "blue"]).resample(time="A").median("time")
composites.ffill("time").bfill("time")

<xarray.DataArray 'stackstac-980122db30eaf66bbdb0a7a3f4f69612' (time: 1,
                                                                band: 3,
                                                                y: 1618, x: 1696)>
dask.array<getitem, shape=(1, 3, 1618, 1696), dtype=float64, chunksize=(1, 1, 894, 936), chunktype=numpy.ndarray>
Coordinates: (12/21)
  * band                         (band) object 'red' 'green' 'blue'
  * x                            (x) float64 4.966e+05 4.966e+05 ... 5.814e+05
  * y                            (y) float64 9.013e+06 9.013e+06 ... 8.932e+06
    platform                     <U9 'landsat-8'
    sci:doi                      <U16 '10.5066/P9OGBGM6'
    landsat:collection_number    <U2 '02'
    ...                           ...
    classification:bitfields     (band) object None None None
    common_name                  (band) object 'red' 'green' 'blue'
    center_wavelength            (band) object 0.65 0.56 0.48
    full_width_half_max          (band) object 0.04 0.06 0.06
    epsg                         int64 32620
  * time                         (time) datetime64[ns] 2017-12-31
Attributes:
    spec:        RasterSpec(epsg=32620, bounds=(496600, 8932050, 581400, 9012...
    crs:         epsg:32620
    transform:   | 50.00, 0.00, 496600.00|\n| 0.00,-50.00, 9012950.00|\n| 0.0...
    resolution:  50

In [17]:
ts = composites.persist()
ts_local = ts.compute()

In [50]:
# define the font size
fontsize = fontSize(ts_local.isel(time=0)[0].to_numpy())
for t in ts_local['time']:
    save_img(ts_local.sel(time=t),fontsize)

Summer 2017


/tmp/ipykernel_49205/495723706.py:76: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  t_width, t_height = fnt.getsize(label)
